<h1>SI 564 Final Project: Trails in U.S. National Parks</h1>
<p>Haley Johnson</p>

<p>Solution key to homework assignment. I used starter code from <a href = "https://gist.github.com/mlhess/be7841fda8baba5645f6aad807b50083">here</a> to connect to SQL.</p>

In [1]:
import os
import pymysql
import pandas as pd

In [2]:
con = pymysql.connect(host='34.134.16.183', port=14192,user='haleyej-ro',password=p,database='nat_parks',cursorclass=pymysql.cursors.DictCursor)  
cur = con.cursor()

<h4>Question 1</h4>
<p>What national park has the longest trail length, on average?</p>

In [3]:
cur.execute('''SELECT p.park, round(avg(t.length), 2) as average_length
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               GROUP BY p.park
               ORDER BY avg(t.length) DESC
               LIMIT 1;''')
for row in cur:
    print(f"{row['park']} National Park, average length: {row['average_length']} yards")

Kings Canyon National Park, average length: 38162.46 yards


<h4>Question 2</h4>
<p>What is the average elevation gain of trails that are in national parks where ‘Mountain,’ ‘Mount’ or any variation of 'Mountain' is in the park's name? </p>

In [4]:
cur.execute('''SELECT p.park, round(avg(t.elevation_gain), 2) as average_elevation_gain
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               WHERE p.park LIKE '%Mountain%' or p.park LIKE '%Mount%'
               GROUP BY p.park
               ORDER BY avg(t.elevation_gain) DESC;''')
for row in cur:
    print(f"{row['park']} National Park\nAverage elevation gain: {row['average_elevation_gain']} yards\n")

Great Smoky Mountains National Park
Average elevation gain: 848.22 yards

Guadalupe Mountains National Park
Average elevation gain: 735.5 yards

Mount Rainier National Park
Average elevation gain: 724.99 yards

Rocky Mountain National Park
Average elevation gain: 590.13 yards



<h4>Question 3</h4>
<p>There are three types of trails in the database. How many of each type of trail is there in Yosemite National Park?</p>

In [5]:
cur.execute('''SELECT r.route_type, count(1) as num_trails
               FROM trails t
               JOIN route_types r
               ON t.route_type_id = r.id
               JOIN parks p 
               ON t.park_id = p.id
               WHERE p.park = "Yosemite"
               GROUP BY r.route_type;''')

for row in cur:
    print(f"There are {row['num_trails']} {row['route_type']} trails")

There are 135 out and back trails
There are 65 loop trails
There are 42 point to point trails


<h4>Question 4</h4>
<p>Alaska has the most national parks out of any state. What are the 5 most popular trails in national parks located in Alaska? </p>

In [6]:
cur.execute('''SELECT p.park, t.name, t.popularity
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               JOIN states s
               ON t.state_id = s.id
               WHERE s.state_name = 'Alaska'
               ORDER BY t.popularity DESC
               LIMIT 5;''')
for row in cur:
    print(f"Trail: {row['name']}\nPark: {row['park']}\nPopularity Rating: {row['popularity']}\n")

Trail: Harding Ice Field Trail
Park: Kenai Fjords
Popularity Rating: 24.8931

Trail: Mount Healy Overlook Trail
Park: Denali
Popularity Rating: 18.0311

Trail: Exit Glacier Trail
Park: Kenai Fjords
Popularity Rating: 17.7821

Trail: Horseshoe Lake Trail
Park: Denali
Popularity Rating: 16.2674

Trail: Triple Lakes Trail
Park: Denali
Popularity Rating: 12.5935



<h4>Question 5</h4>
<p>How many trails are in Yellowstone National Park? </p>

In [7]:
cur.execute('''SELECT count(1) AS number_of_trails
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               WHERE p.park = 'Yellowstone';''')


for row in cur:
    print(f"There are {row['number_of_trails']} trails in Yellowstone National Park")

There are 228 trails in Yellowstone National Park


<h4>Question 6</h4>
<p>Return a list of 5 random trails that are in the contiguous U.S. and have less than 500 feet in elevation gain. </p>

In [8]:
cur.execute('''SELECT p.park, t.name, t.elevation_gain
               FROM trails t
               JOIN parks p 
               ON t.park_id = p.id
               JOIN states s 
               ON t.state_id = s.id
               WHERE s.state_name NOT IN ('Alaska', 'Hawaii')
                     AND t.elevation_gain < (500/3)
               ORDER BY RAND()
               LIMIT 5;''')


for row in cur:
    print(f"Trail: {row['name']}\nPark: {row['park']} National Park\nElevation Gain In Yards: {row['elevation_gain']}\n")

Trail: Black Creek Canal Trail
Park: Biscayne National Park
Elevation Gain In Yards: 0.9144

Trail: Smith Spring Trail
Park: Guadalupe Mountains National Park
Elevation Gain In Yards: 117.9576

Trail: Red Rock Point via Red Point Trail
Park: Yellowstone National Park
Elevation Gain In Yards: 79.8576

Trail: Lehman Caves Visitor Center and Mountain View Nature Trail
Park: Great Basin National Park
Elevation Gain In Yards: 12.8016

Trail: Old Carriage Trail [CLOSED]
Park: Cuyahoga Valley National Park
Elevation Gain In Yards: 105.7656



<h4>Question 7</h4>
<p>Which national park has the most trails with the maximum difficulty rating?  </p>

In [9]:
cur.execute('''SELECT p.park, count(1) AS num_parks
               FROM trails t 
               JOIN parks p 
               ON t.park_id = p.id
               WHERE t.difficulty_rating = 
                (SELECT max(difficulty_rating) FROM trails t)
               GROUP BY p.park
               ORDER BY 2 DESC
               LIMIT 1;''')


for row in cur:
    print(f"{row['park']} National Park has {row['num_parks']} parks with the maximum difficulty rating")

Yosemite National Park has 25 parks with the maximum difficulty rating


<h4>Question 8</h4>
<p>What trails have high average visitor ratings (4.5 or higher) but have the lowerst visitor usage rating</p>

In [10]:
cur.execute('''SELECT t.name, p.park, t.avg_rating, t.visitor_usage
                FROM trails t 
                JOIN parks p
                ON t.park_id = p.id
                WHERE t.avg_rating >= 4.5 
                      AND t.visitor_usage = 
                      (SELECT min(visitor_usage) FROM trails)
                ORDER BY rand()
                LIMIT 10;''')

for row in cur:
    print(f'''{row['name']}\nPark: {row['park']} National Park\nAverage Rating: {row['avg_rating']}\n''')

Waldron Trail
Park: Grand Canyon National Park
Average Rating: 4.5

Coastal Trail: Demartin Section
Park: Redwood National Park
Average Rating: 4.5

Mount Thorofare Ridge Loop
Park: Denali National Park
Average Rating: 4.5

Shuckstack Tower via Appalachian Trail
Park: Great Smoky Mountains National Park
Average Rating: 4.5

Cascade Lake Trail to Observation Peak
Park: Yellowstone National Park
Average Rating: 4.5

California Riding & Hiking Trail
Park: Joshua Tree National Park
Average Rating: 4.5

Redwood Canyon to Redwood Creek
Park: Kings Canyon National Park
Average Rating: 5.0

North Grove Loop Trail
Park: Kings Canyon National Park
Average Rating: 4.5

Beach Trail 2
Park: Olympic National Park
Average Rating: 5.0

Goler Wash and Mengal Pass Route
Park: Death Valley National Park
Average Rating: 5.0



<h4>Question 9</h4>
<p>Give me a list of trails and the parks they're located in that have high difficulty ratings (greater than or equal to 5), but are frequently used by visitors.</p>

<p><b>Note:</b> there is some ambiguity in how answers can define "frequently used." This sample solution conside

In [11]:
cur.execute('''SELECT t.name, p.park, t.difficulty_rating, t.visitor_usage
               FROM trails t 
               JOIN parks p 
               ON t.park_id = p.id
               WHERE t.difficulty_rating >= 5
               AND t.visitor_usage =  
                   (SELECT max(visitor_usage) FROM trails)''')

for row in cur:
    print(f"{row['name']} in {row['park']} National Park\nVisitor Usage Rating (1-4): {row['visitor_usage']}\nDifficulty Rating: {row['difficulty_rating']}\n")
    
    
    

Merced Lake High Sierra Camp Trail in Yosemite National Park
Visitor Usage Rating (1-4): 4.0
Difficulty Rating: 5

Alum Cave Trail to Mount LeConte in Great Smoky Mountains National Park
Visitor Usage Rating (1-4): 4.0
Difficulty Rating: 5

Chimney Tops Trail in Great Smoky Mountains National Park
Visitor Usage Rating (1-4): 4.0
Difficulty Rating: 5

Angels Landing Trail in Zion National Park
Visitor Usage Rating (1-4): 4.0
Difficulty Rating: 5

Old Rag Mountain Loop in Shenandoah National Park
Visitor Usage Rating (1-4): 4.0
Difficulty Rating: 5

Lena Lake Trail to Upper Lena Lake Trail in Olympic National Park
Visitor Usage Rating (1-4): 4.0
Difficulty Rating: 5

